## Get Historical data of instruments

Function to extract the historical data of instruments that include: 

- Date
- Open
- High
- Low
- Close
- Volume
- Change_percent

Values of symbols or columns through which we can search are:

- For stocks: `symbol` column
- For funds: `symbol` column
- For etfs: `symbol` column
- For indices: `symbol` column
- For forex currency pairs: `name` column
- For bonds: `name` column
- For commodities: `name` column
- For certificates: `symbol` column
- For cryptos: `name` column

In [ ]:
# Run and install the dependencies only once.
# ! pip install requests

#### Helpers Functions defined, that will be needed for the main execution.

In [ ]:
import requests
import json

## Extract the json formatted data from the investing xhr request.
def _get_json_(url:str, headers={}, params={}):
    session = requests.session()
    print("Fetching {}..".format(url))
    response = session.get(
        url=url,
        headers=headers,
        params=params
    ).text
    json_response = json.loads(response)
    return json_response

#### Function defined to extract the historical data of instruments

In [ ]:
## imports
import csv
import random
from user_agents import USER_AGENTS

SEARCH_BAR_URL = "https://api.investing.com/api/search/v2/search"
HISTORICAL_DATA_BASE_URL = "https://api.investing.com/api/financialdata/historical/{0}"

common_headers ={
    "user-agent": random.choice(USER_AGENTS)
}

## Function to extract the table of OHLC
def get_historical_data(args_tuple):
    symbol, from_date, to_date = args_tuple[0], args_tuple[1], args_tuple[2]
    params = {
        "q": symbol
    }
    try:
        search_bar_data = _get_json_(
            url=SEARCH_BAR_URL,
            headers=common_headers,
            params=params
        )
        result = search_bar_data["quotes"][0]
        investing_code = result["id"]
        category = result["url"].split("/")[1]
        country = result["flag"]

        ## for data scraping
        output = []
        try:
            params = {
                "start-date": from_date,
                "end-date": to_date,
                "time-frame": "Daily",
                "add-missing-rows": "False"
            }
            response_data = _get_json_(
                url=HISTORICAL_DATA_BASE_URL.format(investing_code),
                headers=common_headers,
                params=params
            )
            # print(response_data)
            historical_data = response_data["data"]
            for item in historical_data:
                output.append({
                    "Date": item["rowDate"],
                    "Open": item["last_open"],
                    "High": item["last_max"],
                    "Low": item["last_min"],
                    "Close": item["last_close"],
                    "Volume": item["volume"],
                    "Change": item["change_precent"]
                })
        except Exception as e:
            output.append({
                "Date": "NA",
                "Open": "NA",
                "High": "NA",
                "Low": "NA",
                "Close": "NA",
                "Volume": "NA",
                "Change": "NA",
            })
            
        field_names = ["Date", "Open", "High", "Low", "Close", "Volume", "Change"]
        with open("{0}-{1}-{2}.csv".format(symbol,category,country), "w", newline="") as csvfile:
            writer = csv.DictWriter(csvfile,fieldnames=field_names)
            writer.writeheader()
            writer.writerows(output)
        return True

    except Exception as e:
        print("{0} symbol not found on investing website..!".format(symbol))

# print(get_historical_data(("SBI","2022-08-01","2022-08-31")))
# print(get_historical_data(("0P0000SBP7","2022-08-01","2022-08-31")))
# print(get_historical_data(("KTPU","2022-08-01","2022-08-31"))) 
# print(get_historical_data(("NIFTYAUTO","2022-08-01","2022-08-31")))


#### Main Function to link the `investing_code` with the searched symbol and saving the output to a csv file

In [ ]:
from datetime import datetime
from concurrent.futures import ThreadPoolExecutor

def main():
    symbols = input("Enter symbols(separated by space): ").split()
    from_date = input("Enter from date(yyyy-mm-dd): ")
    to_date = input("Enter to date(yyyy-mm-dd): ")
    if datetime.strptime(from_date,"%Y-%m-%d").date() > datetime.strptime(to_date,"%Y-%m-%d").date():
        print("Your from_date is greater than the to_date")
    else:
        historical_data_args = []
        for symbol in symbols:
            historical_data_args.append((symbol,from_date,to_date))

        with ThreadPoolExecutor() as executor:
            executor.map(get_historical_data, historical_data_args)


Execution or calling of main function

In [ ]:
# main()